This notebook demonstrates a test of refining the fundamental matrix using the slopes between keypoints.

In [1]:
import os
import pandas as pd
import sys
sys.path.insert(0, os.path.abspath('..'))

import unittest

from autocnet.examples import get_path
from plio.io.io_controlnetwork import to_isis
from plio.io.io_controlnetwork import write_filelist
from autocnet.graph.network import CandidateGraph
from autocnet.utils import utils

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
serial_numbers = {'AS15-M-0295_SML.png': '1971-07-31T01:24:11.754',
                'AS15-M-0296_SML.png': '1971-07-31T01:24:36.970',
                'AS15-M-0297_SML.png': '1971-07-31T01:25:02.243',
                'AS15-M-0298_SML.png': '1971-07-31T01:25:27.457',
                'AS15-M-0299_SML.png': '1971-07-31T01:25:52.669',
                'AS15-M-0300_SML.png': '1971-07-31T01:26:17.923'}

for k, v in serial_numbers.items():
    serial_numbers[k] = 'APOLLO15/METRIC/{}'.format(v)

In [3]:
adjacency = get_path('two_image_adjacency.json')
basepath = get_path('Apollo15')
cg = CandidateGraph.from_adjacency(adjacency, basepath=basepath)

cg.extract_features(method='sift', extractor_parameters={"nfeatures":500})
cg.match_features(k=2)

In [4]:
for source, destination, edge in cg.edges_iter(data=True):
    # Perform the symmetry check
    edge.symmetry_check()
    # Perform the ratio test
    edge.ratio_check(clean_keys=['symmetry'])

/Users/jlaura/anaconda3/envs/autocnet/lib/python3.5/site-packages/pandas/core/frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


In [5]:
cg.apply_func_to_edges("compute_fundamental_matrix", clean_keys=['symmetry', 'ratio'])

In [6]:
f = cg.edge[0][1].fundamental_matrix

In [8]:
df=utils.calculate_slope(f.x1, f.x2)
len(np.asarray(df).ravel())

11

In [9]:
f.mask[f.mask==True].equals(f.mask[f.mask])

True

In [10]:
nbr_inliers = 20
fp = np.array(np.random.standard_normal((nbr_inliers, 2)))  # inliers
static_F = np.array([[4, 0.5, 10], [0.25, 1, 5], [0.2, 0.1, 1]])
fph = np.hstack((fp, np.ones((nbr_inliers, 1))))
tp = static_F.dot(fph.T)
# normalize hom. coordinates
tp /= tp[-1, :np.newaxis]
df1 = pd.DataFrame(fp, columns=['x', 'y'])
df2 = pd.DataFrame(tp.T[:, :2], columns=['x', 'y'])

In [11]:
df1

,x,y
0,1.633181,0.662958
1,-0.652132,0.815878
2,-0.438907,1.047533
3,-0.339436,1.383948
4,-0.958286,0.682298
5,0.235437,-0.778398
6,-0.456880,-1.248944
7,0.235843,-0.454797
8,-0.189979,2.118751
9,0.944133,0.408296


In [12]:
df2

,x,y
0,12.106982,4.358614
1,8.199881,5.943098
2,8.621809,5.838713
3,8.719442,5.884208
4,7.424377,6.209099
5,10.887362,4.416272
6,9.630884,4.640421
7,10.697907,4.596401
8,8.773865,6.023836
9,11.369585,4.590168


In [13]:
mask = df1.x > 0
mask

0      True
1     False
2     False
3     False
4     False
5      True
6     False
7      True
8     False
9      True
10    False
11    False
12     True
13     True
14     True
15     True
16    False
17     True
18     True
19    False
Name: x, dtype: bool

In [14]:
df1[mask]

,x,y
0,1.633181,0.662958
5,0.235437,-0.778398
7,0.235843,-0.454797
9,0.944133,0.408296
12,0.764976,1.178681
13,2.096332,0.580255
14,0.780332,-1.046638
15,0.325916,0.629410
17,0.477673,0.395407
18,1.575866,0.773803


In [15]:
df2[mask]

,x,y
0,12.106982,4.358614
5,10.887362,4.416272
7,10.697907,4.596401
9,11.369585,4.590168
12,10.740136,5.012282
13,12.641683,4.132114
14,11.982098,3.945629
15,10.298838,5.062288
17,10.667482,4.858555
18,11.985440,4.429107


In [16]:
df1[mask] = df2[mask]

In [17]:
df1

,x,y
0,12.106982,4.358614
1,-0.652132,0.815878
2,-0.438907,1.047533
3,-0.339436,1.383948
4,-0.958286,0.682298
5,10.887362,4.416272
6,-0.456880,-1.248944
7,10.697907,4.596401
8,-0.189979,2.118751
9,11.369585,4.590168
